In [1]:
import pandas as pd


In [2]:
df = pd.read_excel('User_record.xlsx')
df

,Full Name,Phone Number,Age,Gender,Address,Email,DOB,Contecting web url
0,Rafi,1.313596e+09,22,Male,ch-2\n,mm@gmail.com,10/12/99,NaN
1,Abir,1.919485e+09,23,Male,ss-02\n,a@gmail.com,3/8/02,NaN
2,Ra,1.212121e+09,21,Female,dd-21\n,R@gmail.com,2/12/98,NaN
3,tasnim,1.211212e+08,22,Female,ch-2\n,t@gmail.com,11/14/99,NaN
4,Tasnim Islam Raisa,NaN,24,Female,"Chadgon A-2, Chittagong\n",NaN,11/14/99,https://www.linkedin.com/in/tasnimislamraisa/
5,jabir,1.313596e+09,22,Male,ccc\n,ja@gmail.com,1/1/00,https://chatgpt.com/c/675aff45-0fe8-8001-9602-...
6,x,1.212596e+09,24,Male,xxx\n,x@gmail.com,1/1/00,https://chatgpt.com/c/675aff45-0fe8-8001-9602-...
7,kabir,1.819339e+09,25,Male,mirpur\n,kabir@gmail.com,1/15/00,https://ostad.app/dashboard/my-courses/6564aea...
8,Mr. sabbir,1.819339e+09,44,Male,Dhaka\n,sabbir@gmail.com,1/25/91,https://github.com/Rafi076/EntryRecord
